In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import plotly.express as px

# Visualize Dataset

## Drug Synergy Data

In [2]:
data_path = "./data/DrugCombDB/"

import os
print(os.getcwd())

# cells: GeneSym (code?), source, protein (id), cell (code), tissue, target_id, weight
df_cell_protein = pd.read_csv(data_path + "cell_protein.csv") 
# combinations: Drug1 (protein?), Drug2 (protein?), cell, drug1_db, drug2_db, synergy
df_drug_combinations = pd.read_csv(data_path + "drug_combinations.csv")
# drug proteins: drug(db format), protein (id)
df_drug_protein = pd.read_csv(data_path + "drug_protein.csv")


/home/julian/Documents/master/swp_cancer/GraphSynergy


In [40]:
df_cell_protein

,GeneSym,source_desc,protein,cell,Tissue,target_id,weight
0,LIMS2,na,55679,HT29,large intestine,-666,1
1,CREB3L1,na,90993,HT29,large intestine,-666,1
2,RHBDF1,na,64285,HT29,large intestine,-666,1
3,HMGA1,na,3159,HT29,large intestine,-666,1
4,PAQR5,na,54852,HT29,large intestine,-666,1
...,...,...,...,...,...,...,...
27725,PIK3R4,na,30849,K562,haematopoietic and lymphoid tissue,-666,-1
27726,ZNF148,na,7707,K562,haematopoietic and lymphoid tissue,-666,-1
27727,USP13,na,8975,K562,haematopoietic and lymphoid tissue,-666,-1
27728,PPP3CA,na,5530,K562,haematopoietic and lymphoid tissue,-666,-1


In [45]:
df_drug_combinations

,Drug1,Drug2,cell,drug1_db,drug2_db,synergy
0,5-FU,BORTEZOMIB,A2058,DB00544,DB00188,-2.3950
1,5-FU,DASATINIB,A2058,DB00544,DB01254,1.5075
2,5-FU,ERLOTINIB,A2058,DB00544,DB00530,8.2525
3,5-FU,GELDANAMYCIN,A2058,DB00544,DB02424,6.0575
4,5-FU,LAPATINIB,A2058,DB00544,DB01259,4.9200
...,...,...,...,...,...,...
69431,Sunitinib malate,Toremifene citrate,HUH7,DB01268,DB00539,1.7320
69432,Sunitinib malate,Aripiprazole,HUH7,DB01268,DB01238,-6.5290
69433,Toremifene citrate,Toremifene citrate,HUH7,DB00539,DB00539,-6.3900
69434,Toremifene citrate,Aripiprazole,HUH7,DB00539,DB01238,-5.5540


In [46]:
df_drug_protein

,drug,protein
0,DB01110,4843
1,DB01234,4843
2,DB08814,4843
3,DB00661,8911
4,DB00909,8911
...,...,...
5285,DB08865,10746
5286,DB08865,11183
5287,DB08865,11213
5288,DB04149,5141


### Histograms

#### Histogram Synergies

In [4]:
print(df_drug_combinations.nlargest(10, 'synergy'))

df_drug_combinations.describe()

                     Drug1        Drug2   cell drug1_db drug2_db   synergy
68705           Fingolimod  RUXOLITINIB  L1236  DB08868  DB08877  620.9770
69402          Doxorubicin   Disulfiram  L1236  DB00997  DB00822  178.9730
69058  PONATINIB (AP24534)   DISULFIRAM  L1236  DB08901  DB00822  175.9135
69056       2-FLUORO ARA-A   DISULFIRAM  L1236  DB01073  DB00822  140.7200
68706           VISMODEGIB  RUXOLITINIB  L1236  DB08828  DB08877  136.7660
68807           GLICLAZIDE  RUXOLITINIB  L1236  DB01120  DB08877  130.3070
68704               EMODIN  RUXOLITINIB  L1236  DB07715  DB08877  127.4465
69055         Flavopiridol   DISULFIRAM  L1236  DB03496  DB00822  126.3840
68800        LEVETIRACETAM  RUXOLITINIB  L1236  DB01202  DB08877   98.2215
68798          flunarizine  RUXOLITINIB  L1236  DB04841  DB08877   83.1990


,synergy
count,69436.000000
mean,-0.051676
std,6.755154
min,-35.420000
25%,-3.420000
50%,-0.460000
75%,2.560000
max,620.977000


In [5]:

fig = px.histogram(df_drug_combinations, x='synergy')
fig.show()

## Protein Interaction Graph

In [6]:
data_path = "./data/DrugCombDB/"

# protein_a, protein_b, data_source(s)
pig_df = pd.read_excel(data_path +  'protein-protein_network.xlsx')

In [7]:
pig_df

,protein_a,protein_b,data_source(s)
0,1,310,"IntAct,PINA"
1,1,368,"IntAct,PINA"
2,1,1026,"IntAct,PINA"
3,1,2886,"IntAct,PINA"
4,1,3958,InnateDB
...,...,...,...
217155,100289678,100289678,IntAct
217156,100506164,100506164,IntAct
217157,100506658,100506658,HPRD
217158,100506742,100506742,BioGRID


In [8]:
protein_graph = nx.from_pandas_edgelist(pig_df, source='protein_a', target='protein_b')

# protein_graph.add_nodes_from([edge for edge in zip(pig_df['protein_a'], pig_df['protein_b'])])
protein_graph.graph

{}

In [9]:
degrees = sorted(list(protein_graph.degree()), key=lambda tup: tup[1])[::-1]
print(degrees)

values = [count for node, count in degrees]
counts, bins = np.histogram(values, bins=range(0, max(values), 5))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'degree of nodes', 'y':'count'})
fig.show()

# now with 95% percentile
values2 = values[round(len(values)*0.05):]
counts, bins = np.histogram(values2, bins=range(0, max(values2), 1))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'degree of nodes', 'y':'count'})
fig.show()

[(7316, 8799), (4899, 2356), (351, 1969), (1994, 1526), (3725, 1407), (6613, 1222), (8452, 1097), (2885, 923), (4609, 880), (1017, 878), (3320, 784), (7341, 777), (7157, 768), (10987, 756), (2335, 743), (1956, 736), (2099, 729), (7534, 726), (6045, 703), (3326, 656), (51547, 652), (10537, 644), (55832, 630), (8454, 629), (22938, 622), (988, 611), (5566, 597), (26270, 589), (3065, 588), (1457, 583), (6714, 557), (4869, 556), (11345, 548), (23710, 517), (7189, 508), (207, 499), (7415, 493), (2033, 490), (2100, 489), (3192, 479), (11337, 478), (3676, 472), (8726, 468), (6117, 458), (120892, 454), (3611, 449), (6118, 448), (2932, 444), (10971, 442), (9531, 439), (7428, 437), (84557, 431), (4193, 430), (5578, 425), (672, 425), (7412, 421), (7532, 419), (4790, 412), (5594, 411), (983, 408), (1499, 404), (8453, 404), (7531, 402), (2534, 395), (8517, 395), (200728, 393), (3066, 392), (4088, 390), (8065, 384), (4087, 383), (4738, 382), (7529, 381), (5071, 380), (6119, 379), (25, 378), (9641, 37

### Important properties of a graph

In [33]:
# Degree Centrality
degree_centrality = pd.Series(nx.degree_centrality(protein_graph))
print(degree_centrality.describe())
print(degree_centrality.nlargest(10))
fig = px.histogram(degree_centrality, title='Degree Centrality')
fig.show()

# Clustering 
avg_clustering = nx.average_clustering(protein_graph)
print('Avg Clustering Coefficient', avg_clustering)

clustering_coefficient = pd.Series(nx.clustering(protein_graph))
print(clustering_coefficient.describe())
print(clustering_coefficient.nlargest(10))
fig = px.histogram(clustering_coefficient, title='Clustering Coefficient')
fig.show()


# Density
print('Density', nx.density(protein_graph))

count    15970.000000
mean         0.001703
std          0.005805
min          0.000063
25%          0.000188
50%          0.000564
75%          0.001691
max          0.551005
dtype: float64
7316    0.551005
4899    0.147536
351     0.123301
1994    0.095560
3725    0.088108
6613    0.076523
8452    0.068696
2885    0.057799
4609    0.055107
1017    0.054982
dtype: float64


Avg Clustering Coefficient 0.20250602005605192
count    15970.000000
mean         0.202506
std          0.234757
min          0.000000
25%          0.022222
50%          0.140242
75%          0.277778
max          1.000000
dtype: float64
1775      1.0
4969      1.0
11250     1.0
64111     1.0
170825    1.0
57091     1.0
221692    1.0
93661     1.0
97        1.0
80148     1.0
dtype: float64


Density 0.0017030491881715251


In [15]:
nx.is_connected(protein_graph)

False

In [20]:
connected_components = sorted(nx.connected_components(protein_graph), key=len, reverse=True)
size_conn_comp = [len(c) for c in connected_components]
print(size_conn_comp)
print(len(size_conn_comp))

[15911, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
48
